> This is a self-correcting activity generated by [nbgrader](https://nbgrader.readthedocs.io). Fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Run subsequent cells to check your code.

---

# Predict diabetes evolution

In this activity, you'll train several regression models to predict the disease progression one year after.

The [Diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) dataset contains ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients, as well as the response of interest, a quantitative measure of disease progression one year after baseline.

## Environment setup

In [1]:
# Import base packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
# Import ML packages
import sklearn

print(f"scikit-learn version: {sklearn.__version__}")

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

scikit-learn version: 0.23.1


## Step 1: Loading the data

In [3]:
dataset = load_diabetes()

# Put data in a pandas DataFrame
df_diab = pd.DataFrame(dataset.data, columns=dataset.feature_names)
# Add target to DataFrame
df_diab["target"] = dataset.target
# Show 10 random samples
df_diab.sample(n=10)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
363,-0.049105,0.050680,-0.024529,0.000068,-0.046975,-0.028245,-0.065491,0.028405,0.019199,0.011349,58.0
110,0.041708,0.050680,-0.032073,-0.022885,-0.049727,-0.040144,0.030232,-0.039493,-0.126097,0.015491,104.0
174,0.016281,0.050680,0.009961,-0.043542,-0.096510,-0.094632,-0.039719,-0.039493,0.017037,0.007207,151.0
346,0.005383,0.050680,0.017506,0.032201,0.127771,0.127390,-0.021311,0.071210,0.062575,0.015491,139.0
129,0.009016,-0.044642,0.016428,0.004658,0.009439,0.010586,-0.028674,0.034309,0.038968,0.119043,268.0
368,-0.016412,0.050680,0.002417,0.014987,0.021822,-0.010082,-0.024993,0.034309,0.085533,0.081764,123.0
35,0.048974,0.050680,-0.030996,-0.049280,0.049341,-0.004132,0.133318,-0.053516,0.021311,0.019633,102.0
295,-0.052738,0.050680,0.039062,-0.040099,-0.005697,-0.012900,0.011824,-0.039493,0.016305,0.003064,85.0
249,-0.012780,-0.044642,0.060618,0.052858,0.047965,0.029375,-0.017629,0.034309,0.070211,0.007207,215.0
41,-0.099961,-0.044642,-0.067641,-0.108957,-0.074494,-0.072712,0.015505,-0.039493,-0.049868,-0.009362,55.0


## Step 2: Preparing the data

### Question

Split the dataset into training (variables `x_train`, `y_train`) and test sets (variables `x_test`, `y_test`) with a 20% ratio.

In [4]:
x = df_diab.drop('target', axis=1)
y = df_diab['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [5]:
print(f"x_train: {x_train.shape}. y_train: {y_train.shape}")
print(f"x_test: {x_test.shape}. y_test: {y_test.shape}")

assert x_train.shape == (353, 10)
assert y_train.shape == (353,)
assert x_test.shape == (89, 10)
assert y_test.shape == (89,)

x_train: (353, 10). y_train: (353,)
x_test: (89, 10). y_test: (89,)


## Step 3: Training several models

In [6]:
def eval_model(model):
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Train and test MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    print(f"Training MSE: {train_mse:.2f}. Test MSE: {test_mse:.2f}")
    
    return train_mse, test_mse

### Question

Create and train a Decision Tree, a MultiLayer Perceptron and a Random Forest on the training data.

Compute their MSE on the training and test data.

In [7]:
# Import the needed sicki-learn packages
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [8]:
# Create and train a Decision Tree

dt_model = DecisionTreeRegressor()
dt_model.fit(x_train, y_train)

eval_model(dt_model)

Training MSE: 0.00. Test MSE: 5776.84


(0.0, 5776.842696629214)

In [9]:
# Create and train a MLP

mlp_model = MLPRegressor()
mlp_model.fit(x_train, y_train)

eval_model(mlp_model)

Training MSE: 23768.54. Test MSE: 21717.47


/home/jchataigner/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


(23768.537163803885, 21717.470218477833)

In [10]:
# Create and train a Random Forest

rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train)
eval_model(rf_model)

Training MSE: 473.22. Test MSE: 3057.98


(473.2208756373938, 3057.9801617977523)

## Step 4: Tuning the most promising model

### Question

Choose the most promising model and tune it, using a `GridSearchCV` instance stored in the `grid_search_cv` variable.

Your test MSE should be less than 3500.

In [11]:
best_model = rf_model
grid_search_cv = GridSearchCV(rf_model, {"max_leaf_nodes": [x for x in range(2,50)], "min_samples_split": [x for x in range(2, 10)]})

In [12]:
# Search for the best parameters with the specified classifier on training data
grid_search_cv.fit(x_train, y_train)

# Print the best combination of hyperparameters found
print(grid_search_cv.best_params_)

{'max_leaf_nodes': 8, 'min_samples_split': 8}


In [13]:
# Evaluate best estimator
train_mse, test_mse = eval_model(grid_search_cv.best_estimator_)

assert train_mse < 1000
assert test_mse < 3500

Training MSE: 2405.78. Test MSE: 3152.21


AssertionError: 